In [33]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import html
import matplotlib as plt
import re
from itertools import combinations
import time
pd.set_option('display.max_columns', None)

In [34]:
historic_df=pd.read_csv('cleaned_historic_inspections.csv')

In [35]:
historic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14428 entries, 0 to 14427
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   objectid                                  14428 non-null  int64  
 1   globalid                                  14428 non-null  object 
 2   Seating Interest (Sidewalk/Roadway/Both)  14428 non-null  object 
 3   Restaurant Name                           14428 non-null  object 
 4   Legal Business Name                       14428 non-null  object 
 5   Doing Business As (DBA)                   14427 non-null  object 
 6   Building Number                           14329 non-null  object 
 7   Street                                    14428 non-null  object 
 8   Borough                                   14428 non-null  object 
 9   Postcode                                  14428 non-null  int64  
 10  Business Address                  

In [36]:
historic_df['Doing Business As (DBA)'].fillna(historic_df['Restaurant Name'], inplace=True)
historic_df['Column Of Interest'] = historic_df['Restaurant Name'] + " " + historic_df['Street']
historic_df['Global Restaurant ID'] = ''
historic_df['Latest Inspection Row'] = ''
historic_df['BIN'] = historic_df['BIN'].astype(str).replace('nan', 'undefined').replace('nan', 'undefined').fillna('undefined')

In [37]:
historic_df['BIN'].value_counts()

undefined    1402
3000000.0      29
4000000.0      21
3397861.0      18
3000090.0      17
             ... 
4124422.0       1
2013298.0       1
1035447.0       1
4311574.0       1
1025423.0       1
Name: BIN, Length: 8960, dtype: int64

In [45]:
#Function to fill Global Restaurant ID
global_restaurant_id_counter = 0
def assign_global_restaurant_id(df, unique_column, threshold=85):
    global global_restaurant_id_counter
    # Filter rows where 'Global Restaurant ID' is NaN
    filtered_df = df[df['Global Restaurant ID'] == '']

    # Iterate through rows in the group
    for bin_value in filtered_df['BIN'].unique():
        # Group rows by BIN
        bin_group = filtered_df[filtered_df['BIN'] == bin_value]

        # Iterate through rows in the BIN group
        for index, row in bin_group.iterrows():
            # Check if the 'Column of Interest' values are similar
            if pd.isna(filtered_df.loc[index, 'Global Restaurant ID']) or filtered_df.loc[index, 'Global Restaurant ID'] == '':
                # Check if the 'Column of Interest' values are similar with other rows in the group
                match = bin_group[bin_group.index != index]['Column Of Interest'].apply(lambda x: fuzz.ratio(row['Column Of Interest'], x) > threshold)

                # If there are matches, assign the same global restaurant ID to the current row and matching rows
                if match.any():
                    global_id = f"{global_restaurant_id_counter:08d}"
                    filtered_df.at[index, 'Global Restaurant ID'] = global_id
                    filtered_df.loc[match.index[match==True], 'Global Restaurant ID'] = global_id
                    global_restaurant_id_counter += 1
                else:
                    # If no match, assign the next global restaurant ID to the single row
                    filtered_df.at[index, 'Global Restaurant ID'] = f"{global_restaurant_id_counter:08d}"
                    global_restaurant_id_counter += 1
        
        # Merge the result_df with the original df
    df = pd.merge(df, filtered_df[[unique_column, 'Global Restaurant ID']], on=unique_column, how='left', suffixes=('', '_filtered'))
    df['Global Restaurant ID'] = df['Global Restaurant ID_filtered'].combine_first(df['Global Restaurant ID'])
    df.drop(columns=['Global Restaurant ID_filtered'], inplace=True)

    return df



historic_df = assign_global_restaurant_id(historic_df,'globalid',threshold=85)

In [47]:
#Function to find the latest Inspection Row
def find_latest_inspection_row(df, time_column):
    df[time_column] = pd.to_datetime(df[time_column], errors='coerce')
    latest_indices = df.groupby('Global Restaurant ID')[time_column].idxmax()
    df.loc[latest_indices, 'Latest Inspection Row'] = 1
    return df

historic_df = find_latest_inspection_row(historic_df,'Time of Submission')

In [48]:
historic_df

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Column Of Interest,Global Restaurant ID,Latest Inspection Row
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965.0,TW,no,NaN,yes,2022-06-09 16:26:00,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville,Madame Bonte East 84th Street,00000000,1
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,Sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526.0,RW,no,NaN,yes,2020-06-26 20:38:00,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South,Seasoned Vegan St. Nicholas Avenue,00000001,1
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,Sidewalk,Americas Cafe & Grill,68th Grill Inc.,Americas Cafe & Grill,1159,3rd Avenue,Manhattan,10065,"1159 3rd Avenue, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,2021-10-22 11:01:00,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island,Americas Cafe & Grill 3rd Avenue,00000002,1
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,Roadway,Sushi Seki,Seki Inc.,Sushi Seki,208,West 23rd Street,Manhattan,10011,"208 West 23rd Street, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,0.0,OP,no,NaN,yes,2020-12-14 19:54:00,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square,Sushi Seki West 23rd Street,00000004,
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,Roadway,Sami & Susu,Amir Nathan,Sami & Susu,190,Orchard Street,Manhattan,10002,"190 Orchard Street, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,0.0,TW,no,NaN,yes,2021-12-22 12:46:00,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown,Sami & Susu Orchard Street,00000005,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14429,9799,108C6784-54DB-480B-8627-CF0495D27A0E,Both,King Of Spades Inc.,King Of Spades Inc.,King Of Spades Inc.,1425,College Point Boulevard,Queens,11356,"1425 College Point Boulevard, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,0.0,OP,no,NaN,yes,2020-08-04 15:27:00,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point,King Of Spades Inc. College Point Boulevard,00012038,1
14430,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,Both,La Queen Tea House Inc.,La Queen Tea House Inc.,La Queen Tea House Inc.,752B,61st Street,Brooklyn,11220,"752B 61st Street, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,2020-07-13 15:53:00,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East,La Queen Tea House Inc. 61st Street,00011494,1
14431,2895,476F88D2-D3E8-4E83-8686-8BAC2F752CF5,Sidewalk,Aahar Indina Cuisine,Vidhan Bhatt Inc.,Vidhan Bhatt Inc.,10,Murray Street,Manhattan,10007,"10 Murray Street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882.0,RW,no,NaN,yes,2020-06-21 13:46:00,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy,Aahar Indina Cuisine Murray Street,

In [50]:
historic_df['Global Restaurant ID'].value_counts()

00002253    8
00003640    7
00001620    7
00004557    6
00003164    6
           ..
00001331    1
00006398    1
00000505    1
00006399    1
00012039    1
Name: Global Restaurant ID, Length: 12040, dtype: int64

In [53]:
historic_df[historic_df['Global Restaurant ID']=='00000000']

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Column Of Interest,Global Restaurant ID,Latest Inspection Row
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965.0,TW,no,NaN,yes,2022-06-09 16:26:00,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville,Madame Bonte East 84th Street,00000000,1
1253,13165,CCC23E5F-7C57-43AA-869D-071A348FABFF,Sidewalk,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,10.0,5.0,50.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,2022-01-12 13:10:00,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville,Madame Bonte East 84th Street,00000000,
7528,13416,190B3F21-87A1-4A42-8F5D-EA69A6B42CE5,Sidewalk,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,15.0,5.0,75.0,NaN,NaN,NaN,yes,no,yes,0.0,TW,no,NaN,yes,2022-04-20 10:10:00,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville,Madame Bonte East 84th Street,00000000,


In [ ]:
result_df[result_df['Restaurant Name']=='Love And Dough']

In [ ]:
historic_df[historic_df['BIN']=='undefined']

In [ ]:
fuzz.ratio("Love And Dough Pearl Street", "Love And Dough Jay Street")

In [ ]:
inspections_df=pd.read_csv('cleaned_current_open_inspections.csv')

In [ ]:
inspections_df

In [ ]:
dohmh_df=pd.read_csv('cleaned_dohmh.csv')

In [ ]:
dohmh_df

In [ ]:
#I'm not sure if OLAP database is the correct option for this since we can't cleanly combine all 3.